# Bag of Words: Traditional Aprroach

In [1]:
import os
import pandas as pd
import re
import numpy as np
from khmernltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from google.colab import drive
from gensim.models import FastText
from gensim.test.utils import common_texts


### Function to clean and tokenize text (Preprocessing)
- Remove spcial symbols
- Tokenize by split the words with space

In [2]:
# Function to clean and tokenize full article text
def clean_and_tokenize(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Filter tokens: keep words, numbers, and common punctuation (.,!?)
    cleaned_tokens = []
    for token in tokens:
        # Allow Khmer text (letters), numbers, spaces, and punctuation (.,!?)
        if re.match(r'^[\u1780-\u17FF"]+$', token):  # Khmer Unicode range + numbers and some punctuation
            cleaned_tokens.append(token)

    # Join the cleaned tokens and remove multiple spaces
    cleaned_text = ' '.join(cleaned_tokens)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

Load Data from folder

In [3]:
documents = []
file_names = []
training_data = 'clean_data1'

- Load the data into documents list for later embedding step
- Extract the Summary and Full Article and put in dataframe for easy access

In [ ]:
import os

documents = []

for file in sorted(os.listdir(training_data)):
    if file.endswith(".txt"):
        file_path = os.path.join(training_data, file)

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        summary = None
        full_article = None

        for i, line in enumerate(lines):
            if line.strip() == "Summary:":
                summary = lines[i + 1].strip()
            elif line.strip() == "Full Article:":
                full_article = ''.join(lines[i + 1:]).strip()
                break

        if summary and full_article:
            article_name = os.path.splitext(file)[0]  # Remove ".txt"
            documents.append({
                'name': article_name,
                'summary': clean_and_tokenize(summary),
                'full_text': clean_and_tokenize(full_article),
            })
            file_names.append(file)
        else:
            print(f"Missing summary or full article in {file}. Skipping.")


| 2025-06-18 19:51:55,326 | INFO | khmer-nltk | Loaded model from /Users/mac/Desktop/thesis/.venv/lib/python3.11/site-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav |


In [ ]:
# df = pd.DataFrame(documents)
# df.to_csv('text_data.csv')

In [ ]:
df = pd.DataFrame(documents)
df

,Unnamed: 0,name,summary,full_text
0,0,article_1,ស្ត្រី ជា បុគ្គល ដ៏ សំខាន់ ក្នុង រួមចំណែក ដំណើ...,កាលពី អតីតកាល កិច្ចការងារ និង មុខ តំណែង ក្នុង ...
1,1,article_100,អ្នកនាំពាក្យ ក្រសួង ការបរទេស អាម៉េរិក លោក កាលព...,ដំណើរទស្សនកិច្ច ទៅកាន់ ទីក្រុង សេអ៊ូល ជា ផ្នែក...
2,2,article_1000,ព្រឹទ្ធសភា អាមេរិក នៅ ថ្ងៃទី ៩ ខែកុម្ភៈ បាន យល...,សវនាការ ដក តំណែង លើក ទី ពីរ ប្រឆាំង អតីត ប្រធា...
3,3,article_10000,សកម្មភាព លក់ លំនៅឋាន ដែល មាន ផ្ទះល្វែង វីឡា ភ្...,របាយការណ៍ ដែល យោង តាម និយ័តករ អាជីវករ អចលន វត្...
4,4,article_10001,ភ្នំពេញ៖ លោក វ៉ាង យី រដ្ឋមន្រ្តី ការបរទេស ចិន ...,ក្នុង ជំនួប នេះ លោក សុខ ចិន្តា សោភា បាន រំលេច ...
...,...,...,...,...
10020,10020,article_9995,របាយការណ៍ ថ្មី មួយ ដែល សិក្សា ដោយ អង្គការ លីកា...,យោង តាម របាយការណ៍ នេះ អ្នក ខ្ចី ប្រាក់ ជាង ៣ យ...
10021,10021,article_9996,៧ខែ មក នេះ កម្ពុជា ទទួលបាន ភ្ញៀវ ទេសចរ បរទេស ស...,ក្រសួង ទេសចរណ៍ បាន បញ្ជាក់ ថា បន្ទាប់ពី ថៃ ទេស...
10022,10022,article_9997,រយៈពេល ៧ខែ មក នេះ ការនាំចេញ របស់ កម្ពុជា ទៅកាន...,គឺជា កិច្ចព្រម ពៀង ភាពជាដៃគូ សេដ្ឋកិច្ច គ្រប់ ...
10023,10023,article_9998,ធនាគារ ជាតិ នៃ កម្ពុជា ប្រកាស ដាក់ ដេញថ្លៃ ប្រ...,បើ តាម សេចក្តីជូនដំណឹង របស់ ធនាគារ ជាតិ ការដេញ...


Split data into train and test set

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

Train Bag of Words to learn the words Then Embedd the entire documents

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(train['full_text'])

X = vectorizer.transform(df['full_text'])

print("Vocabulary:", vectorizer.get_feature_names_out())
# Print the Bag-of-Words matrix
print("BoW Representation:")
print(X.toarray())

Vocabulary: ['កក' 'កកក' 'កកករល' ... '៩៩៦' '៩៩៨' '៩៩៩ស']
BoW Representation:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


Evaluate the model performance

In [ ]:
# Step 3: Retrieval@10 accuracy using summaries and 'name' as label
correct_in_top10 = 0
not_in = 0

for _, row in test.iterrows():
    summary_vec = vectorizer.transform([row['summary']])  # BoW vector for summary
    similarities = cosine_similarity(summary_vec, X)[0]    # Similarity to all data docs
    top_indices = similarities.argsort()[::-1][:10]        # Top 10 most similar

    # Get top 10 predicted article names from training set
    top_names = df.iloc[top_indices]['name'].values

    # Check if the correct article name is among top 10
    if row['name'] in top_names:
        correct_in_top10 += 1
    else:
        not_in += 1

accuracy = correct_in_top10 / len(test)
print(f"Khmer BoW Retrieval@10 Accuracy: {accuracy * 100:.2f}")
print(not_in)

Khmer BoW Retrieval@10 Accuracy: 58.05
841


In [ ]:
top_names

array(['article_1881', 'article_644', 'article_1005', 'article_4919',
       'article_6789', 'article_1477', 'article_4412', 'article_2925',
       'article_6726', 'article_8540'], dtype=object)